In [ ]:
from bs4 import BeautifulSoup
import urllib 
import re 
import pandas as pd 
from IPython.core.display import display, HTML 
from wordcloud import WordCloud # to plot wordclouds 
import matplotlib.pyplot as plt
from urllib.request import urlopen

plt.style.use('ggplot')
%matplotlib inline


df_songs = pd.read_csv("billboard.csv",encoding= 'utf-8')
df_pop =df_songs.drop(['Pop','Country'], axis=1)

#df_pop.to_csv("songs_list.csv")

df_artists = pd.read_csv("artist_combined.csv",encoding= 'utf-8')
songs_list = list(df_songs.RnB)
year_list = list(df_songs.Year)
artists_list = list(df_artists.RnB)
combo =[]

for i in range(len(songs_list)):
    combo.append(f"{artists_list[i]}:{songs_list[i]}".replace('\xa0',' '))
    
lyric_dict = {}

for item in combo:
    url_search = ('http://lyrics.wikia.com/wiki/Special:Search?query='+ item)
    
    site = urlopen(url_search.replace(' ', '+')).read()
    soup = BeautifulSoup(site)
    links = []
    
    for link in soup.find_all("a", class_='result-link'):
        try:
            if link.get('href') != None:
                links.append(link.get('href'))                
                print ('Getting lyrics from...', links[0])
        
            song_url = links[0]
            print(song_url)        
        except:
            pass
    
    try:
        site = urlopen(song_url).read()
        soup = BeautifulSoup(site)
        lyric = soup.find_all("div", class_="lyricbox")

        if len(lyric) > 0:
            for element in lyric:
                lyric_dict[item] = re.sub("([a-z])([A-Z])","\g<1> \g<2>", BeautifulSoup(str(element).replace('<br/>',' ')).get_text())

                print (re.sub("([a-z])([A-Z])","\g<1> \g<2>", BeautifulSoup(str(element).replace('<br/>',' ')).get_text()))
    except:
        pass

In [ ]:
# Account for missing lyrics
year_list =list(df_songs.Year)
year_list.pop(69)
year_list.pop(63)
year_list.pop(55)
year_list.pop(23)
year_list.pop(0)

In [ ]:
def plot_word_cloud(song_name,corpus, max_words = 42, width=600, height=400, fig_size=(8,6)):
    try:
        if len(corpus) == 0:
            corpus = 'no words'
        wordcloud = WordCloud(max_words = max_words, width=width, height=height, background_color="black").generate(corpus)
        plt.figure(figsize=fig_size, dpi=80, facecolor = 'black',edgecolor="black")
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.title(str(year_list.pop(0)) + " : " + artist_name+"\n" + "'"+song_name+ "'", color='white',backgroundcolor ='black')
        plt.savefig(song_name + ".jpg",bbox_inches='tight', pad_inches = 0)        
        plt.show()
        return
    except:
        pass
    return
    
for tune in lyric_dict:
    artist_name,song_name=tune.split(":")
    plot_word_cloud(song_name,str(lyric_dict[tune]).lower(), max_words=50, width=400, height=250)